# Verkefni 2, Battleship
## Karl Sigurvinsson - kas76@hi.is

Til þess að búa til borðið í þessari úfærslu af Battleship er notast við fallið fyllaBord(), sem býr til nýtt tvívítt 10x10 fylki og fyllir það af "skipum" af lengd = 5 reitir. Skipin eru sett niður af handahófi með því að fyrst velja af handahófi hvort skipið eigi að liggja lárétt eða lóðrétt, og svo finna slembin x og y gildi sem passa þar við. Þar þarf að hafa í huga að ef skipið liggur lárétt, þá má y gildið ekki vera stærra en 5 svo það fari ekki út fyrir borðið, og svo það sama með að x má ekki vera yfir 5 þegar skipið er lóðrétt. \
Þar sem skipin eru sett niður af handahófi er notast við fallið skarast() sem kemur í veg fyrir að skipið skarist við annað skip, það er gert með því að ítra í gegnum staðsetningu skipsins áður en það er sett niður, ef árekstur finnst þarf að byrja upp á nýtt með ný upphafsgildi. \
\
Aðferðin til þess að lágmarka fjölda sprengja virkar þannig að við skiptum borðinu upp í fjóra hluta: við byrjum efst í hólfi [0,0] og vinnum okkur ská niður til hægri, þegar við erum kominn neðst niður til hægri í hólf [9,9] byrjum við aftur í hólfi [5,0] og gerum það sama niður í hólf [9,4] og byrjum aftur í hólfi [0,5] og gerum það sama. Þannig erum við að skipta upp borðinu á þann hátt að ekkert skip getur verið falið á borðinu. Í hvert skipti sem við hittum á skip á leiðinni vinnum við okkur í gegnum það með því að sprengja okkur upp, niður, hægri, vinstri eins og við á þangað til við erum búin með skipið og byrjum þá aftur á staðnum sem við vorum komin á, eins og gert er í "næst heimskustu" aðferðinni. \
Með því að skipta borðinu svona upp sést að það getur ekki verið neitt skip sem verður ekki fyrir skoti, án þess að skjóta af handahófi á borðið eins og í "næst heimskustu" aðferðinni, sem tekur út þó nokkra reiti á borðinu sem við þurfum ekki að spá í. \
Eins og sést á prufum að þá lágmarkar þetta fjöldann að meðaltali ef við miðum við aðferðina sem "Sendir sprengjur á handahófsstaði í grindinni, en um leið og það verður sprenging þá leita á skynsamlegri hátt að restinni af skipinu (þ.e. prófa út eftir línu eða dálki) og klára að sökkva því."
\
\
Hér fyrir neðan er forritið sjálft:


In [0]:
# main

sum = 0
for i in range(20):
  sum += randomBomb()
sum = sum/20
print("meðalfjöldi sprengja fyrir heimskustu aðferðina: ", sum)

sum = 0
for i in range(20):
  sum += searchBomb()
sum = sum/20
print("meðalfjöldi sprengja fyrir skynsamlegri aðferðina: ", sum)

sum = 0
for i in range(20):
  sum += betriKostur()
sum = sum/20
print("meðalfjöldi sprengja fyrir bestu aðferðina: ", sum)

meðalfjöldi sprengja fyrir heimskustu aðferðina:  94.5
meðalfjöldi sprengja fyrir skynsamlegri aðferðina:  39.15
meðalfjöldi sprengja fyrir bestu aðferðina:  28.8


In [0]:

import random

# aðferðin sem skiptir upp borðinu í fjóra parta og gefur bestu lausn:
def betriKostur():
  # búa til hreint borð til að fylgjast með hvað er búið að bomba:
  bord2 = [[" " for i in range(10)]for j in range(10)]
  bord = fyllaBord()
  # byrja með x og y í [0,0]
  x = y = bombur = 0
  skipHP = 15
  while(True):
    uX = x
    uY = y
    if(bord2[x][y] != "X"):
      skipHP, s = bomba(bord,x,y,skipHP)
      bombur += 1
      if(s == "hit"):
        bord2[x][y] = "O"
        skipHP, bombur = sprengjaSkip(x,y,bord,bord2,skipHP,bombur)
        x = uX
        y = uY
      else: bord2[x][y] = "X"
    if(skipHP<=0):
      prentaBord(bord)
      return bombur
    # þegar við erum komin neðst í hægra horn byrjum við aftur í [5,0] og
    # vinnum okkur niður til hægri:
    elif(y == 9 and x == 9):
      x = 5
      y = 0
    # þegar við erum komin í hólf [9,4] förum við upp í hólf [0,5] og
    # vinnum okkur aftur niður til hægri:
    elif(x == 9 and y == 4):
      x = 0
      y = 5
    # hækkum alltaf x og y um 1, förum alltaf niður til hægri:
    else:
      x += 1
      y += 1  
  


# aðferð sem bombar bara random þangað til öll skip eru sokkin
def randomBomb():
  # búa til hreint borð til að fylgjast með hvað er búið að bomba:
  bord2 = [[" " for i in range(10)]for j in range(10)]
  bord = fyllaBord()
  bombur = 0
  skipHP = 15
  while(skipHP>0):
    # finna random gildi fyrir x og y:
    x = random.randrange(0,10)
    y = random.randrange(0,10)
    # tékka hvort búið sé að bomba staðsetninguna:
    if(bord2[x][y] != "X"):
      bord2[x][y] = "X"
      skipHP, s = bomba(bord,x,y,skipHP)
      bombur += 1
  prentaBord(bord)
  #print(bombur)
  return bombur

# aðferð sem sprengir random en bætir sig ef skip finnst
def searchBomb():
  # til að sjá hvort búið sé að sprengja
  bord2 = [[" " for i in range(10)]for j in range(10)]
  bord = fyllaBord()
  bombur = 0
  skipHP = 15
  while(skipHP>0):
    x = random.randrange(0,10)
    y = random.randrange(0,10)
    if(bord2[x][y] != "X"):
      bord2[x][y] = "X"
      skipHP, s = bomba(bord,x,y,skipHP)
      bombur += 1
      if(s == "hit"):
        skipHP, bombur = sprengjaSkip(x,y,bord,bord2,skipHP,bombur) 
  prentaBord(bord)
  #print(bombur)
  return bombur


print("munurinn á borðunum (versta fyrst):")
randomBomb()
searchBomb()
betriKostur()

munurinn á borðunum (versta fyrst):
-----------------------------------------
|   | X | X | O | X | O | X | X | X | X |
-----------------------------------------
| X | X |   | O | X | O | X |   | X | X |
-----------------------------------------
| X | X | X | O | X | O | X | X | X | X |
-----------------------------------------
| X | X | X | O | X | O | X |   | X | X |
-----------------------------------------
| X | X | X | O | X | O |   | X | X |   |
-----------------------------------------
| X | X | X | X | O | O | O | O | O | X |
-----------------------------------------
| X | X | X | X |   | X | X | X | X | X |
-----------------------------------------
| X | X | X | X | X | X | X | X | X | X |
-----------------------------------------
| X | X | X | X | X | X | X | X | X | X |
-----------------------------------------
| X | X | X | X | X | X | X | X | X | X |
-----------------------------------------
-----------------------------------------
| X | X |   |   | X |   |   | X |   | X 

31

In [0]:

import random

def bomba(bord,x,y,HP):
  if(bord[x][y]==("1" or "O")):
    bord[x][y] = "O"
    HP -= 1
    s = "hit"
    return HP, s
  else:
    bord[x][y] = "X"
    s = "miss"
  return HP, s

def sprengjaSkip(x,y,bord,bord2,HP,bombur):
  # geyma upphaflegu gildin:
  uX = x
  uY = y
  skot = 0
  
  # tékkum hvort við hittum skipið aftur með því að
  # skjóta upp, niður, hægri og vinstri
  # ef við hittum höldum við áfram
  while(x>0):
    x -= 1
    if(bord2[x][y]!= "X"):
      bord2[x][y] = "X"
      HP, s = bomba(bord,x,y,HP)
      if(s == "hit"):
        skot += 1
        bombur += 1
        continue
      else: break
    else: continue
  x = uX
  while(x<9):
    x += 1
    if(bord2[x][y]!= "X"):
      bord2[x][y] = "X"
      HP, s = bomba(bord,x,y,HP)
      if(s == "hit"):
        skot += 1
        bombur += 1
        continue
      else: break
    else: continue
  x = uX
  while(y>0):
    y -= 1
    if(bord2[x][y]!= "X"):
      bord2[x][y] = "X"
      HP, s = bomba(bord,x,y,HP)
      if(s == "hit"):
        skot += 1
        bombur += 1
        continue
      else: break
    else: continue
  y = uY
  while(y<9):
    y += 1
    if(bord2[x][y]!= "X"):
      bord2[x][y] = "X"
      HP, s = bomba(bord,x,y,HP)
      if(s == "hit"):
        skot += 1
        bombur += 1
        continue
      else: break
    else: continue
  return HP, bombur



In [0]:
import numpy as np
import random


# fylla borð af skipum, þrjú skip öll með lengd 5
def fyllaBord():

  bord = [[" " for i in range(10)]for j in range(10)]
  i = 0
  while(i != 3):
    # tékka hvort skip eigi að vera lárétt eða lóðrétt:
    lalo = np.random.binomial(1, .5)
    if(lalo==1):
      x = random.randrange(0, 5)
      y = random.randrange(0,10)
    else:
      x = random.randrange(0,10)
      y = random.randrange(0,5)
    # tékka hvort skipin skarist:
    if(skarast(x, y, bord, lalo)):
      i -= 1
    else:  
      for j in range(5):
        bord[x][y] = "1"
        if(lalo==1):
          x += 1
        else:
          y += 1
    i += 1 
  #prentaBord(bord)
  #temp = bord
  return bord

def skarast(x,y,bord, lalo):
  for i in range(5):
    if(bord[x][y]=="1"):
      return True
    elif(lalo==1):
      x += 1
    else:
      y += 1

def prentaBord(bord):
  for i in range(len(bord)):
    print("-----------------------------------------")
    for j in range(len(bord[0])):
      print("| " + bord[i][j] + " ", end = '')
    print("|", end = '')
    print("")
  print("-----------------------------------------")

#fyllaBord()

